# Exercises I - Creating Dataframe from SQL

- Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine)

- cd into your exercises folder for this module and run echo env.py >> .gitignore

- Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

- Use your function to obtain a connection to the employees database.

- Once you have successfully run a query:

- a. Intentionally make a typo in the database url. What kind of error message do you see?

- b. Intentionally make an error in your SQL query. What does the error message look like?

In [98]:
import env
import pandas as pd

db = 'employees'

url = env.db_url(db)

In [99]:
# 5. Successfully run a query

pd.read_sql('select * from employees limit 1', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26


In [100]:
# 6. Read the employees and titles tables into two separate DataFrames.

employees = pd.read_sql('SELECT * FROM employees', url)

In [101]:
titles = pd.read_sql('SELECT * FROM titles', url)

In [102]:
# 7. How many rows and columns do you have in each DataFrame?

employees.shape # (300024, 6)

(300024, 6)

In [103]:
titles.shape # (443308, 4)

(443308, 4)

In [104]:
# 8. Display the summary statistics for each DataFrame.

employees.describe()

# employees.drop(columns='emp_no').describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [105]:
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


In [106]:
# 9. How many unique titles are in the titles DataFrame?

titles['title'].value_counts().count() # 7

7

In [107]:
# 10. What is the oldest date in the to_date column?

titles.to_date.min() # 1985-03-01

datetime.date(1985, 3, 1)

In [108]:
# 11. Convert the 'to_date' column to datetime
titles['to_date'] = pd.to_datetime(titles['to_date'], errors='coerce')

# errors = 'coerce' turns any errors into NaT (Not a time)

titles['to_date'].max()

Timestamp('2002-08-01 00:00:00')

# Exercises II - Indexing, Subsetting, Aggregating, Merging, and Joining

In [136]:
# 1. Copy the users and roles DataFrames from the examples above.

import numpy as np
import pandas as pd

# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})

# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})

print(users, '\n''\n', roles)


   id   name  role_id
0   1    bob      1.0
1   2    joe      2.0
2   3  sally      3.0
3   4   adam      3.0
4   5   jane      NaN
5   6   mike      NaN 

    id       name
0   1      admin
1   2     author
2   3   reviewer
3   4  commenter


In [145]:
# 2. What is the result of using a right join on the DataFrames?

# Complete right table is preserved, causing NaN on the left.

users.merge(roles, left_on='role_id', right_on='id', how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [143]:
# 3. What is the result of using an outer join on the DataFrames?

# All NaN are preserved.

users.merge(roles, left_on='role_id', right_on='id', how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


In [112]:
# 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

# Performs a concat essentially since there are no matches to link

In [113]:
# 5. Load the mpg dataset from PyDataset.

from pydataset import data

mpg = data('mpg')

In [114]:
# 6. Output and read the documentation for the mpg dataset.

data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [115]:
# 7. How many rows and columns are in the dataset?

mpg.shape

# 234 rows, 11 columns

(234, 11)

In [116]:
# 8. Check out your column names and perform any cleanup you may want on them.

mpg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


In [142]:
# 9. Display the summary statistics for the dataset.

# Numbers described
print(mpg.describe())

# Objects described
print(mpg.loc[:, mpg.dtypes == 'object'].describe())

            displ         year         cyl         cty         hwy  \
count  234.000000   234.000000  234.000000  234.000000  234.000000   
mean     3.471795  2003.500000    5.888889   16.858974   23.440171   
std      1.291959     4.509646    1.611534    4.255946    5.954643   
min      1.600000  1999.000000    4.000000    9.000000   12.000000   
25%      2.400000  1999.000000    4.000000   14.000000   18.000000   
50%      3.300000  2003.500000    6.000000   17.000000   24.000000   
75%      4.600000  2008.000000    8.000000   19.000000   27.000000   
max      7.000000  2008.000000    8.000000   35.000000   44.000000   

       mileage_difference  average_mileage  
count          234.000000       234.000000  
mean             6.581197        20.149573  
std              2.262739         5.050290  
min              2.000000        10.500000  
25%              5.000000        15.500000  
50%              7.000000        20.500000  
75%              8.000000        23.500000  
max      

In [118]:
# 10. How many different manufacturers are there?

print(pd.Series(pd.unique(mpg['manufacturer'])))

len(pd.Series(pd.unique(mpg['manufacturer'])))

pd.crosstab(mpg['manufacturer'], mpg['manufacturer'].count())

0           audi
1      chevrolet
2          dodge
3           ford
4          honda
5        hyundai
6           jeep
7     land rover
8        lincoln
9        mercury
10        nissan
11       pontiac
12        subaru
13        toyota
14    volkswagen
dtype: object


col_0,234
manufacturer,
audi,18
chevrolet,19
dodge,37
ford,25
honda,9
hyundai,14
jeep,8
land rover,4
lincoln,3


In [119]:
# 11. How many different models are there?

print(pd.Series(pd.unique(mpg['model'])))

len(pd.Series(pd.unique(mpg['model'])))

0                         a4
1                 a4 quattro
2                 a6 quattro
3         c1500 suburban 2wd
4                   corvette
5            k1500 tahoe 4wd
6                     malibu
7                caravan 2wd
8          dakota pickup 4wd
9                durango 4wd
10       ram 1500 pickup 4wd
11            expedition 2wd
12              explorer 4wd
13           f150 pickup 4wd
14                   mustang
15                     civic
16                    sonata
17                   tiburon
18        grand cherokee 4wd
19               range rover
20             navigator 2wd
21           mountaineer 4wd
22                    altima
23                    maxima
24            pathfinder 4wd
25                grand prix
26              forester awd
27               impreza awd
28               4runner 4wd
29                     camry
30              camry solara
31                   corolla
32    land cruiser wagon 4wd
33         toyota tacoma 4wd
34            

38

In [120]:
# 12. Create a column named mileage_difference like you did in the DataFrames exercises;
# this column should contain the difference between highway and city mileage for each car.

mpg['mileage_difference'] = mpg['hwy'] - mpg['cty']

mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


In [121]:
# 13. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

mpg['average_mileage'] = (mpg['cty'] + mpg['hwy']) / 2

mpg.head(2)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0


In [122]:
# 14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

# Could also do a .contains string method
# mpg['trans'].str.contains('auto')

mpg['trans'] = mpg['trans'].apply(lambda x: 'auto' if 'auto' in x else 'manual')

mpg['is_automatic'] = mpg['trans'] == 'auto'

mpg.head(2)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto,f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual,f,21,29,p,compact,8,25.0,False


In [148]:
# 15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

mfr = mpg.groupby('manufacturer').average_mileage.mean()

print(mfr) 

mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending=False).head(1)

manufacturer
audi          22.027778
chevrolet     18.447368
dodge         15.540541
ford          16.680000
honda         28.500000
hyundai       22.750000
jeep          15.562500
land rover    14.000000
lincoln       14.166667
mercury       15.625000
nissan        21.346154
pontiac       21.700000
subaru        22.428571
toyota        21.720588
volkswagen    25.074074
Name: average_mileage, dtype: float64


,manufacturer,average_mileage
0,honda,28.5


In [147]:
# 16. Do automatic or manual cars have better miles per gallon?

mpg.groupby('trans').average_mileage.mean().reset_index()

,trans,average_mileage
0,auto,19.130573
1,manual,22.227273


# Exercises III - Reshaping and Transposing

In [137]:
# 1. Use your get_db_url function to help you explore the data from the chipotle database.

import env
import pandas as pd

url = env.db_url('chipotle')

chipdf = pd.read_sql('SELECT * from orders', url)

chipdf

#Clean item_price
chipdf['item_price'] = chipdf['item_price'].str.replace('$', '').astype(float)

chipdf

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39
1,2,1,1,Izze,[Clementine],3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75


In [126]:
# 2 What is the total price for each order?

chipdf.groupby('order_id')['item_price'].sum()

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

In [148]:
# 3. What are the most popular 3 items?

chipdf['item_name'].value_counts().head(3)

# pd.crosstab(chipdf['item_name'],['quantity']).sort_values('quantity', ascending=False).head(3)

# chipdf.groupby('item_name').count().sort_values('quantity', ascending=False)[['quantity']].head(3)

item_name
Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: count, dtype: int64

In [150]:
# 4. Which item has produced the most revenue?

# Chicken Bowl

chipdf.groupby('item_name').sum()['item_price'].sort_values(ascending=False).head(1)

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

In [129]:
# 5. Join the employees and titles DataFrames together.

import env
import pandas as pd

# Get employees table

url1 = env.db_url('employees')

emp = pd.read_sql('SELECT * from employees', url1)

In [130]:
# Get titles table

url2 = env.db_url('employees')

titles = pd.read_sql('SELECT * FROM titles', url2)

In [131]:
# Merge the tables

merged = pd.merge(emp, titles, how='left', on='emp_no')

merged.head(5)

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


In [152]:
# 6. For each title, find the hire date of the employee that was hired most recently with that title.

merged['hire_date'] = pd.to_datetime(merged['hire_date'])
merged.groupby('title').apply(lambda x: x.loc[x['hire_date'].idxmax(), ['hire_date', 'emp_no', 'first_name', 'last_name']])

,hire_date,emp_no,first_name,last_name
title,,,,
Assistant Engineer,1999-12-24,243702,Munehiro,Luke
Engineer,2000-01-28,463807,Bikash,Covnot
Manager,1992-02-05,110420,Oscar,Ghazalie
Senior Engineer,2000-01-01,108201,Mariangiola,Boreale
Senior Staff,2000-01-13,222965,Volkmar,Perko
Staff,2000-01-12,47291,Ulf,Flexer
Technique Leader,1999-12-31,294732,Karlis,Orsini


In [133]:
# 7. Write the code necessary to create a cross tabulation of the number of titles by department.
# (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

url = env.db_url('employees')

titles_by_dept = pd.read_sql('''
SELECT t.title, d.dept_name
FROM titles t
JOIN dept_emp de ON t.emp_no = de.emp_no
JOIN departments d ON de.dept_no = d.dept_no
''', url)

# Create a cross tabulation of the number of titles by department
titles_by_dept_crosstab = pd.crosstab(titles_by_dept['dept_name'], titles_by_dept['title'])

# Print the resulting DataFrame
print(titles_by_dept_crosstab)

title               Assistant Engineer  Engineer  Manager  Senior Engineer  \
dept_name                                                                    
Customer Service                   298      2362        4             2027   
Development                       7769     58135        2            49326   
Finance                              0         0        2                0   
Human Resources                      0         0        2                0   
Marketing                            0         0        2                0   
Production                        6445     49649        4            42205   
Quality Management                1831     13852        4            11864   
Research                           378      2986        2             2570   
Sales                                0         0        2                0   

title               Senior Staff  Staff  Technique Leader  
dept_name                                                  
Customer Service     

In [134]:
# 7. Write the code necessary to create a cross tabulation of the number of titles by department.
# (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

url = env.db_url('employees')

titles = pd.read_sql('''
                     SELECT *
                     FROM titles''',
                     url)

department = pd.read_sql('''
                         SELECT *
                         FROM departments''',
                         url)

dept_emp = pd.read_sql('''
                       SELECT *
                       FROM dept_emp''',
                       url)

squish = pd.merge(dept_emp, department, how='left', on='dept_no')

all_three = pd.merge(squish, titles, how='left', on='emp_no')

# Create a cross tabulation of the number of titles by department
crosstab_titles = pd.crosstab(all_three['dept_name'], all_three['title'])

# Print the resulting DataFrame
print(crosstab_titles)

title               Assistant Engineer  Engineer  Manager  Senior Engineer  \
dept_name                                                                    
Customer Service                   298      2362        4             2027   
Development                       7769     58135        2            49326   
Finance                              0         0        2                0   
Human Resources                      0         0        2                0   
Marketing                            0         0        2                0   
Production                        6445     49649        4            42205   
Quality Management                1831     13852        4            11864   
Research                           378      2986        2             2570   
Sales                                0         0        2                0   

title               Senior Staff  Staff  Technique Leader  
dept_name                                                  
Customer Service     

# Random Practice Code